# Create Document-Term Matrices
* No pre-processing
* Remove stop words
* Remove stop words + tf-idf
* Stem
* Stem + Stop Words
* Stem + Stop + Tf-IDF

## Create full df

In [2]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import Normalizer


from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import spacy

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/site-packages/sklearn/utils/__init__.py:4: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sequence


In [3]:
clean_filepath = "/Users/kylieleblancKylie/domino/docsim/data/clean/"

In [4]:
df_corpus = pd.read_csv(clean_filepath + 'text_transcripts.csv')
df_corpus = df_corpus.set_index('doc')
df_corpus.sample(5)

,text,year,semester,scenario
doc,,,,
2019_84_5C_Transcript.docx,Yeah. How are you feeling in general? Jus...,2018-19,spring,behavior
68_c_Transcript.docx,Nice. What did you think? Yeah? What ...,2017-18,fall,feedback
01_1920_05_085_22C_Transcript.docx,"So, how do you feel that went? How so? ...",2019-20,fall,behavior
2019_80_5C_Transcript.docx,"Well, it definitely showed. But I do – eve...",2018-19,spring,behavior
59_c_Transcript.docx,How do you feel? You got through? ...,2017-18,fall,feedback


In [5]:
df_ideal = pd.read_csv(clean_filepath + 'text_scripts.csv')
df_ideal = df_ideal.set_index('doc')
df_ideal.sample(3)

,text,scenario
doc,,
Feedback Model B22.docx,How are you feeling about that first simulati...,feedback
Feedback Model C22.docx,Begin Transcript 3 How are you feeling about ...,feedback
Feedback Model B12.docx,How are you feeling about that first simulati...,feedback


In [6]:
df = df_ideal.append(df_corpus, sort = True)
df.sample(5)

,scenario,semester,text,year
doc,,,,
2018_61_3C_Transcript.docx,feedback,fall,Okay so I am going to start our timer for f...,2018-19
32-2C.docx,behavior,spring,And how are you feeling about that? The firs...,2017-18
93-2C.docx,behavior,spring,"Come on I said Um, so how are you feeling? ...",2017-18
102-2C.docx,behavior,spring,"Okay. Yeah, you want-- Yeah. So, general...",2017-18
01_1920_05_107_22c_Transcript.docx,behavior,fall,This is Rosalie Chung’s Coaching Session fo...,2019-20


In [6]:
df[df.text.isnull()]

,scenario,semester,text,year
doc,,,,


# Doc-Term Matrix

In [7]:
docs = list(df.text)
vec = CountVectorizer()
X = vec.fit_transform(docs)
matrix = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix.columns))
matrix.sample()

Number of words:  3502


,00,04,05,10,100,12,13,15,16,18,...,young,younger,your,yours,yourself,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
01_1920_05_101_22C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0


In [8]:
matrix_ideal = matrix[matrix.index.str.contains('Model')]
matrix_ideal.to_csv(clean_filepath + 'matrix_scripts.csv')
matrix_ideal.sample(5)

,00,04,05,10,100,12,13,15,16,18,...,young,younger,your,yours,yourself,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
Classroom Management Model B3.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,3,0,0,0,0,0,0,0
Feedback Model C22.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,0,0
Classroom Management Model C6.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,0,0,0
Classroom Management Model C3.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,5,0,0,0,0,0,0,0
Feedback Model D11.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,8,0,0,0,0,0,0,0


In [9]:
matrix_corpus = matrix[~matrix.index.str.contains('Model')]
matrix_corpus.to_csv(clean_filepath + 'matrix_transcripts.csv')
matrix_corpus.sample()

,00,04,05,10,100,12,13,15,16,18,...,young,younger,your,yours,yourself,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
2018_26_3C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Weight

In [10]:
docs = list(df.text)
vec = TfidfVectorizer()
X = vec.fit_transform(docs)
matrix_wgt = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_wgt.columns))
matrix_wgt.sample()

matrix_ideal_wgt = matrix_wgt[matrix_wgt.index.str.contains('Model')]
matrix_ideal_wgt.to_csv(clean_filepath + 'matrix_scripts_wgt.csv')
matrix_ideal_wgt.sample()


matrix_corpus_wgt = matrix_wgt[~matrix_wgt.index.str.contains('Model')]
matrix_corpus_wgt.to_csv(clean_filepath + 'matrix_transcripts_wgt.csv')
matrix_corpus_wgt.sample()

Number of words:  3502


,00,04,05,10,100,12,13,15,16,18,...,young,younger,your,yours,yourself,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
88-2C.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.020517,0.0,0.030087,0.0,0.0,0.0,0.0,0.0


# Remove Stop Words (no weighting)

In [11]:
stop_words = list(stopwords.words('english'))
stop_words.append('like')
stop_words.append('um')
stop_words.append('uhm')
stop_words.append('-pron-')
stop_words

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [12]:
docs = list(df.text)
vec = CountVectorizer(stop_words = stop_words)
X = vec.fit_transform(docs)
matrix_stop = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_stop.columns))
matrix_stop.sample()

Number of words:  3367


,00,04,05,10,100,12,13,15,16,18,...,yet,yo,york,young,younger,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
37_c_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
matrix_ideal_stop = matrix_stop[matrix_stop.index.str.contains('Model')]
matrix_ideal_stop.to_csv(clean_filepath + 'matrix_scripts_stop.csv')
matrix_ideal_stop.sample()

,00,04,05,10,100,12,13,15,16,18,...,yet,yo,york,young,younger,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
Classroom Management Model C3.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
matrix_corpus_stop = matrix_stop[~matrix_stop.index.str.contains('Model')]
matrix_corpus_stop.to_csv(clean_filepath + 'matrix_transcripts_stop.csv')
matrix_corpus_stop.sample()

,00,04,05,10,100,12,13,15,16,18,...,yet,yo,york,young,younger,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
87_c_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Stop and weight

In [15]:
docs = list(df.text)
vec = TfidfVectorizer(stop_words = stop_words)
X = vec.fit_transform(docs)
matrix_stop_wgt = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_stop_wgt.columns))
matrix_stop_wgt.sample()

matrix_ideal_stop_wgt = matrix_stop_wgt[matrix_stop_wgt.index.str.contains('Model')]
matrix_ideal_stop_wgt.to_csv(clean_filepath + 'matrix_scripts_stop_wgt.csv')
matrix_ideal_stop_wgt.sample()


matrix_corpus_stop_wgt = matrix_stop_wgt[~matrix_stop_wgt.index.str.contains('Model')]
matrix_corpus_stop_wgt.to_csv(clean_filepath + 'matrix_transcripts_stop_wgt.csv')
matrix_corpus_stop_wgt.sample()

Number of words:  3367


,00,04,05,10,100,12,13,15,16,18,...,yet,yo,york,young,younger,yup,zen,zero,zeroing,zone
doc,,,,,,,,,,,,,,,,,,,,,
01_1920_05_008_22C_Transcript.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Stem

In [16]:
# Initialize spacy 'en' model, keeping only tagger component needed for lemmatization
nlp = spacy.load('en', disable=['parser', 'ner'])

sentence = "The striped bats are hanging on their feet for best singing."

# Parse the sentence using the loaded 'en' model object `nlp`
doc = nlp(sentence)

# Extract the lemma for each token and join
" ".join([token.lemma_ for token in doc])
#> 'the strip bat be hang on -PRON- foot for good'

'the strip bat be hang on -PRON- foot for good singing .'

In [17]:
stems = [" ".join([token.lemma_ for token in nlp(text)]) for text in df.text]
df['text'] = stems
df.sample(5)

,scenario,semester,text,year
doc,,,,
Classroom Management Model C1.docx,behavior,NaN,how be -PRON- feel about that first simulation...,NaN
70-2C.docx,behavior,spring,-PRON- know about that . good . ye...,2017-18
62_c_Transcript.docx,feedback,fall,"so , what be some thing that -PRON- feel l...",2017-18
2018_69_3C_Transcript.docx,feedback,fall,okay so -PRON- be go to start the timer her...,2018-19
93-2C.docx,behavior,spring,"come on -PRON- say um , so how be -PRON- ...",2017-18


In [18]:
docs = list(df.text)
vec = CountVectorizer()
X = vec.fit_transform(docs)
matrix_stem = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_stem.columns))
matrix_stem.sample()

Number of words:  2697


,00,04,05,10,100,12,13,15,16,18,...,yo,york,you,young,your,yours,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
18-2C.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0


In [19]:
matrix_scripts_stem = matrix_stem[matrix_stem.index.str.contains('Model')]
matrix_scripts_stem.to_csv(clean_filepath + 'matrix_scripts_stem.csv')
matrix_scripts_stem.sample()

,00,04,05,10,100,12,13,15,16,18,...,yo,york,you,young,your,yours,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
Feedback Model A12.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
matrix_transcripts_stem = matrix_stem[~matrix_stem.index.str.contains('Model')]
matrix_transcripts_stem.to_csv(clean_filepath + 'matrix_transcripts_stem.csv')
matrix_transcripts_stem.sample()

,00,04,05,10,100,12,13,15,16,18,...,yo,york,you,young,your,yours,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
2019_47_5C_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Stop and Stem

In [21]:
docs = list(df.text)
vec = CountVectorizer(stop_words = stop_words)
X = vec.fit_transform(docs)
matrix_stem_stop = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_stem_stop.columns))
matrix_stem_stop.sample()

Number of words:  2582


,00,04,05,10,100,12,13,15,16,18,...,yes,yesterday,yet,yo,york,young,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
01_1920_05_115_22c_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0


In [22]:
matrix_scripts_stem_stop = matrix_stem[matrix_stem_stop.index.str.contains('Model')]
matrix_scripts_stem_stop.to_csv(clean_filepath + 'matrix_scripts_stem_stop.csv')
matrix_scripts_stem_stop.sample()

,00,04,05,10,100,12,13,15,16,18,...,yo,york,you,young,your,yours,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
Feedback Model C12.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [23]:
matrix_transcripts_stem_stop = matrix_stem[~matrix_stem_stop.index.str.contains('Model')]
matrix_transcripts_stem_stop.to_csv(clean_filepath + 'matrix_transcripts_stem_stop.csv')
matrix_transcripts_stem_stop.sample()

,00,04,05,10,100,12,13,15,16,18,...,yo,york,you,young,your,yours,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
51_c_Transcript.docx,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Stop Stem and Weight

In [24]:
docs = list(df.text)
vec = TfidfVectorizer(stop_words = stop_words)
X = vec.fit_transform(docs)
matrix_stem_stop_wgt = pd.DataFrame(X.toarray(), columns=vec.get_feature_names(), index = df.index)
print('Number of words: ', len(matrix_stem_stop_wgt.columns))
matrix_stem_stop_wgt.sample()

Number of words:  2582


,00,04,05,10,100,12,13,15,16,18,...,yes,yesterday,yet,yo,york,young,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
2018_57_3C_Transcript.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.015004,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [25]:

matrix_scripts_stem_stop_wgt = matrix_stem_stop_wgt[matrix_stem_stop_wgt.index.str.contains('Model')]
matrix_scripts_stem_stop_wgt.to_csv(clean_filepath + 'matrix_scripts_stem_stop_wgt.csv')
matrix_scripts_stem_stop_wgt.sample()


,00,04,05,10,100,12,13,15,16,18,...,yes,yesterday,yet,yo,york,young,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
Feedback Model A12.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
matrix_transcripts_stem_stop_wgt = matrix_stem_stop_wgt[~matrix_stem_stop_wgt.index.str.contains('Model')]
matrix_transcripts_stem_stop_wgt.to_csv(clean_filepath + 'matrix_transcripts_stem_stop_wgt.csv')
matrix_transcripts_stem_stop_wgt.sample()

,00,04,05,10,100,12,13,15,16,18,...,yes,yesterday,yet,yo,york,young,yup,zen,zero,zone
doc,,,,,,,,,,,,,,,,,,,,,
107-2C.docx,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.033387,0.0,0.0,0.0


# LSA

In [43]:
matrix_transcripts_lsa = pd.read_csv(clean_filepath + 'matrix_transcripts.csv')
matrix_scripts_lsa = pd.read_csv(clean_filepath + 'matrix_scripts.csv')
matrix_lsa = matrix_transcripts_lsa.append(matrix_scripts_lsa)
matrix_lsa = matrix_lsa.set_index('doc')

lsa = TruncatedSVD(n_components = 100, random_state = 100)
lsa_fit = lsa.fit_transform(matrix_lsa)
lsa_fit = Normalizer(copy=False).fit_transform(lsa_fit)
print(lsa_fit.shape)

## Each LSA component is a linear combo of words
word_weights = pd.DataFrame(lsa.components_, columns = matrix_lsa.columns)
word_weights.head()

## Each document is a linear combination of components
matrix_lsa = pd.DataFrame(lsa_fit, index = matrix_lsa.index, columns = word_weights.index)
matrix_lsa.sample(5)


matrix_scripts_lsa = matrix_lsa[matrix_lsa.index.str.contains('Model')]
matrix_scripts_lsa.to_csv(clean_filepath + 'matrix_scripts_lsa.csv')

matrix_transcripts_lsa = matrix_lsa[~matrix_lsa.index.str.contains('Model')]
matrix_transcripts_lsa.to_csv(clean_filepath + 'matrix_transcripts_lsa.csv')

(324, 100)


# LSA + Weights

In [44]:
matrix_transcripts_lsa_wgt = pd.read_csv(clean_filepath + 'matrix_transcripts_wgt.csv')
matrix_scripts_lsa_wgt = pd.read_csv(clean_filepath + 'matrix_scripts_wgt.csv')
matrix_lsa_wgt = matrix_transcripts_lsa_wgt.append(matrix_scripts_lsa_wgt)
matrix_lsa_wgt = matrix_lsa_wgt.set_index('doc')

lsa = TruncatedSVD(n_components = 100, random_state = 100)
lsa_fit = lsa.fit_transform(matrix_lsa_wgt)
lsa_fit = Normalizer(copy=False).fit_transform(lsa_fit)
print(lsa_fit.shape)

## Each LSA component is a linear combo of words
word_weights = pd.DataFrame(lsa.components_, columns = matrix_lsa_wgt.columns)
word_weights.head()

## Each document is a linear combination of components
matrix_lsa_wgt = pd.DataFrame(lsa_fit, index = matrix_lsa_wgt.index, columns = word_weights.index)
matrix_lsa_wgt.sample(5)


matrix_scripts_lsa_wgt = matrix_lsa_wgt[matrix_lsa_wgt.index.str.contains('Model')]
matrix_scripts_lsa_wgt.to_csv(clean_filepath + 'matrix_scripts_lsa_wgt.csv')

matrix_transcripts_lsa_wgt = matrix_lsa_wgt[~matrix_lsa_wgt.index.str.contains('Model')]
matrix_transcripts_lsa_wgt.to_csv(clean_filepath + 'matrix_transcripts_lsa_wgt.csv')

(324, 100)


# LSA + Stop

In [8]:
matrix_transcripts_lsa_stop = pd.read_csv(clean_filepath + 'matrix_transcripts_stop.csv')
matrix_scripts_lsa_stop = pd.read_csv(clean_filepath + 'matrix_scripts_stop.csv')
matrix_lsa_stop = matrix_transcripts_lsa_stop.append(matrix_scripts_lsa_stop)
matrix_lsa_stop = matrix_lsa_stop.set_index('doc')

lsa = TruncatedSVD(n_components = 100, random_state = 100)
lsa_fit = lsa.fit_transform(matrix_lsa_stop)
lsa_fit = Normalizer(copy=False).fit_transform(lsa_fit)
print(lsa_fit.shape)

## Each LSA component is a linear combo of words
word_weights = pd.DataFrame(lsa.components_, columns = matrix_lsa_stop.columns)
word_weights.head()

## Each document is a linear combination of components
matrix_lsa_stop = pd.DataFrame(lsa_fit, index = matrix_lsa_stop.index, columns = word_weights.index)
matrix_lsa_stop.sample(5)


matrix_scripts_lsa_stop = matrix_lsa_stop[matrix_lsa_stop.index.str.contains('Model')]
matrix_scripts_lsa_stop.to_csv(clean_filepath + 'matrix_scripts_lsa_stop.csv')

matrix_transcripts_lsa_stop = matrix_lsa_stop[~matrix_lsa_stop.index.str.contains('Model')]
matrix_transcripts_lsa_stop.to_csv(clean_filepath + 'matrix_transcripts_lsa_stop.csv')

(324, 100)


# LSA, Weighting, Stop Words

In [45]:
matrix_transcripts_lsa_wgt_stop = pd.read_csv(clean_filepath + 'matrix_transcripts_stop_wgt.csv')
matrix_scripts_lsa_wgt_stop = pd.read_csv(clean_filepath + 'matrix_scripts_stop_wgt.csv')
matrix_lsa_wgt_stop = matrix_transcripts_lsa_wgt_stop.append(matrix_scripts_lsa_wgt_stop)
matrix_lsa_wgt_stop = matrix_lsa_wgt_stop.set_index('doc')

lsa = TruncatedSVD(n_components = 100, random_state = 100)
lsa_fit = lsa.fit_transform(matrix_lsa_wgt_stop)
lsa_fit = Normalizer(copy=False).fit_transform(lsa_fit)
print(lsa_fit.shape)

## Each LSA component is a linear combo of words
word_weights = pd.DataFrame(lsa.components_, columns = matrix_lsa_wgt_stop.columns)
word_weights.head()

## Each document is a linear combination of components
matrix_lsa_wgt_stop = pd.DataFrame(lsa_fit, index = matrix_lsa_wgt_stop.index, columns = word_weights.index)
matrix_lsa_wgt_stop.sample(5)


matrix_scripts_lsa_wgt_stop = matrix_lsa_wgt_stop[matrix_lsa_wgt_stop.index.str.contains('Model')]
matrix_scripts_lsa_wgt_stop.to_csv(clean_filepath + 'matrix_scripts_lsa_wgt_stop.csv')

matrix_transcripts_lsa_wgt_stop = matrix_lsa_wgt_stop[~matrix_lsa_wgt_stop.index.str.contains('Model')]
matrix_transcripts_lsa_wgt_stop.to_csv(clean_filepath + 'matrix_transcripts_lsa_wgt_stop.csv')

(324, 100)
